In [1]:
import pandas as pd
import numpy as np
import re
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import CPTAC.Ovarian as CPTAC

Loading Ovarian CPTAC data:
Loading somatic_19 data...
Loading clinical data...
Loading cnv data...
Loading proteomics data...
Loading transcriptomics data...
Loading somatic_38 data...
Loading phosphoproteomics data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


In [7]:
somatic = CPTAC.get_somatic_mutations()
data = CPTAC.get_data()

In [39]:
data[5]

,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,ExAC_AC_AN_Adj,ExAC_AC_AN,ExAC_AC_AN_AFR,ExAC_AC_AN_AMR,ExAC_AC_AN_EAS,ExAC_AC_AN_FIN,ExAC_AC_AN_NFE,ExAC_AC_AN_OTH,ExAC_AC_AN_SAS,ExAC_FILTER
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
PRAMEF17,0,.,GRCh38,chr1,13392142,13392142,+,Silent,SNP,G,...,40/36804,97/75150,1/744,2/5228,2/3392,2/3152,31/19784,0/230,2/4274,PASS
ZCCHC17,0,.,GRCh38,chr1,31346774,31346774,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS
BAI2,0,.,GRCh38,chr1,31756152,31756152,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STIL,0,.,GRCh38,chr1,47251736,47251736,+,Silent,SNP,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IFI44L,0,.,GRCh38,chr1,78628280,78628280,+,Missense_Mutation,SNP,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AP4B1,0,.,GRCh38,chr1,113897883,113897883,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
POGZ,0,.,GRCh38,chr1,151428073,151428073,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PEAR1,0,.,GRCh38,chr1,156907610,156907610,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CD84,0,.,GRCh38,chr1,160553453,160553453,+,Missense_Mutation,SNP,C,...,40/106174,40/106210,2/9062,7/11212,0/7862,0/6614,21/54338,0/694,10/16392,PASS


In [15]:
CPTAC.get_proteomics().transpose()

refseq_peptide,NP_000005,NP_000007,NP_000008,NP_000009,NP_000010,NP_000012,NP_000013,NP_000014,NP_000017,NP_000018,...,YP_003024026,YP_003024027,YP_003024028,YP_003024029,YP_003024030,YP_003024031,YP_003024032,YP_003024035,YP_003024036,YP_003024038
hgnc_symbol,A2M,ACADM,ACADS,ACADVL,ACAT1,PSEN1,ADA,SGCA,ADSL,AGA,...,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-ATP8,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB
C01OV007,0.76512,-0.228234,0.356409,0.180033,-0.348284,-0.24445,0.275867,NaN,0.181865,-1.09838,...,-0.179725,-0.498896,-0.823043,0.0873713,-0.220362,-0.350893,-0.310186,0.160645,-0.194748,-0.163941
C01OV017,-0.461906,-0.16933,-0.659938,-0.584018,-0.0759776,0.41212,-0.756128,NaN,-0.0452906,-1.5965,...,0.781057,0.065106,-0.665079,0.538887,0.315786,-0.0418409,0.211591,0.690928,0.626079,0.320735
C01OV018,0.0658422,-0.0517297,0.947164,0.456308,-0.250954,-0.225354,0.547258,NaN,0.255754,-0.897808,...,0.0839202,0.730063,-0.728606,0.106007,0.179399,0.156049,-0.357757,0.179438,-0.186853,-0.432308
C01OV023,1.51208,-0.294907,-0.399714,0.0742397,-0.465211,-0.0507748,0.46335,NaN,0.567784,-2.14068,...,-0.235595,-0.364906,-0.11072,-0.311199,0.323893,-0.129739,0.668505,0.0137165,-0.537361,-0.6413
C01OV026,0.343872,-0.306999,-0.249258,-0.0432278,-0.293727,0.336951,0.0286104,NaN,0.510154,-0.0209876,...,-0.238203,0.519368,-0.312152,0.0713594,0.124463,0.120169,-0.736425,-0.132413,-0.359127,-0.711696
C01OV029,-0.312811,0.109829,-0.312841,-0.199112,-0.115732,0.143854,2.07044,NaN,0.749544,-1.55854,...,0.405532,0.189853,0.0698987,0.601288,-0.0381608,0.518939,NaN,0.256534,0.192244,0.0146161
C01OV030,-0.338799,-0.684895,-0.168461,-0.210199,-0.704224,-0.0171958,0.618801,NaN,-0.401158,NaN,...,-0.212004,0.406973,-0.371703,-0.0953367,0.241845,0.34103,-0.179221,-0.482215,0.160153,-0.537567
C01OV039,-0.492472,-0.322297,-0.215577,0.424556,0.0393394,0.000325669,-0.683557,NaN,0.467531,-0.468974,...,0.781046,0.543519,-0.995797,0.630569,0.500559,0.682067,-0.166798,0.298753,0.509268,1.47654
C01OV041,-0.523344,-0.540556,-0.0344378,0.0659294,-0.317093,-0.448147,-0.199382,NaN,-0.185703,-0.79599,...,0.246061,0.480461,0.987988,0.669851,-0.219528,0.158004,0.793036,0.533034,-0.193313,0.0976903
